In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import numpy as np

In [29]:
class BC:
    def __init__(self):
        self.x_min=None
        self.x_max=None
        self.y_min=None
        self.y_max=None
        self.f_r=None
        self.f_l=None
        self.f_u=None
        self.f_d=None
    
    def random_data(self):
        x_rand = np.random.uniform(low=self.x_min,
                                   high=self.x_max,
                                   size=1)[0]
        y_rand = np.random.uniform(low=self.y_min,
                                   high=self.y_max,
                                   size=1)[0]
        type_index= np.random.randint(4,size=1)[0]

        if(type_index==0):
            return np.array([x_rand,self.y_max,self.f_u(x_rand,self.y_max)])
        elif(type_index==1):
            return np.array([self.x_max,y_rand,self.f_r(self.x_max,y_rand)])
        elif(type_index==2):
            return np.array([x_rand,self.y_min,self.f_d(x_rand,self.y_min)])
        else:
            return np.array([self.x_min,y_rand,self.f_l(self.x_min,y_rand)])
        
    def get_data(self,size):
        result = np.array([self.random_data()])
        for _ in range(size-1):
            result = np.vstack([result,self.random_data()])
        return result
    
    def get_dimension(self):
        return 2

In [30]:
class BCbuilder:
    def __init__(self):
        self.bc = BC()

    def set_x_domain(self,x_domain):
        self.bc.x_min = x_domain[0]
        self.bc.x_max = x_domain[1]
        return self

    def set_y_domain(self,y_domain):
        self.bc.y_min = y_domain[0]
        self.bc.y_max = y_domain[1]
        return self

    def set_f_Right(self,f_r):
        self.bc.f_r=f_r
        return self

    def set_f_Left(self,f_l):
        self.bc.f_l=f_l
        return self

    def set_f_Down(self,f_d):
        self.bc.f_d=f_d
        return self

    def set_f_Up(self,f_u):
        self.bc.f_u=f_u
        return self
    
    def build(self):
        return self.bc

In [ ]:
class chnagableBC(BC):
    def __init__(self):
        super().__init__()
        self.f_list=[]

    def change(self):
        random_index = np.random.randint(len(self.f_list),size=1)[0]
        self.f_r = self.f_list[random_index]['f_r']
        self.f_l = self.f_list[random_index]['f_l']
        self.f_u = self.f_list[random_index]['f_u']
        self.f_d = self.f_list[random_index]['f_d']

    def copyBC(self):
        bc = BCbuilder().set_x_domain([self.x_min,self.x_max])\
        .set_y_domain([self.y_min,self.y_max])\
        .set_f_Right(self.f_r)\
        .set_f_Left(self.f_l)\
        .set_f_Down(self.f_d)\
        .set_f_Up(self.f_u)\
        .build()
        return bc

In [ ]:
class ChangableBCbuilder(BCbuilder):
    def __init__(self):
        self.bc=chnagableBC()

    def add_f_dict(self,f_dict):
        self.f_list.append(f_dict)
        return self
    
    def build(self):
        return self.bc

In [ ]:
class mainNet(nn.Module):
    def __init__(self, subnetwork):
        super(mainNet, self).__init__()
        self.sub = subnetwork
        self.node_size = [3,5,7,9,7,5,3]
        self.layer_list=[]
        self.act = nn.Tanh()
        for i,j in zip(self.node_size[:-1],self.node_size[1:]):
            self.layer_list.append(nn.Linear(i*2,j))
            self.layer_list.append(self.act)
        self.layer_list.append(nn.Linear(self.node_size[-1]*2,1))
        self.hidden_layers = nn.ModuleList(self.layer_list)

        self.sub.set_output_layer(self)

    def forward(self, x):
        batch_size = x.size()[0]
        bc_info = self.sub(batch_size)
        bc_data = bc_info[0]
        bc_list = bc_info[1]
        for i,layer in enumerate(self.hidden_layers):
            x = layer(x,bc_data[i]) # 합치는 코드 필요
        return [x,bc_list]

In [ ]:
class subNet(nn.Module):
    def __init__(self,BC,size):
        super(subNet,self).__init__()
        self.boundary = BC
        self.n_bc = size
        self.layer_list=[]
        self.act = nn.ReLU()
        for i in range(self.boundary.get_dimension(),1,-1):
            self.layer_list.appned(nn.Linear(size * i, size * (i-1)))
            self.layer_list.append(self.act)
        self.return_size=None
        self.hidden_layers=None

    def forward(self,n_batch):
        bc_list=[]
        x = np.array([])
        for _ in range(n_batch):
            x = np.vstack([x,self.boundary.get_data(self.n_bc).reshape(-1)])
            bc_list.append(self.boundary.copyBC())
            self.boundary.change()

        x = torch.from_numpy(x)

        x = self.hidden_layers(x)

        x_split = []
        start_index = 0
        for i in self.return_size[1]:
            x_split.append(x[:,start_index:start_index+i])
            start_index += i
        
        return [x_split,bc_list]

    def read_output_type(self,module):
        result = 0
        for i in range(module.node_size):
            result += i
        return [result,module.node_size]

    def set_output_layer(self,module):
        self.return_size = self.read_output_type(module)
        self.layer_list.append(nn.Linear(self.n_bc,self.return_size[0]))
        self.hidden_layers = nn.Sequential(*self.layer_list)